In [1]:
from torchtext.datasets import IMDB

train_iter = IMDB(split='train')

/Users/mkhokhlush/github/pytorch-ml-experiments/.venv/lib/python3.10/site-packages/torch/utils/data/datapipes/utils/common.py:24: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  warnings.warn(
/Users/mkhokhlush/github/pytorch-ml-experiments/.venv/lib/python3.10/site-packages/torch/utils/data/datapipes/iter/selecting.py:54: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  warnings.warn("Lambda function is not supported for pickle, please use "


In [2]:
for label, line in train_iter:
    print(label, line)
    break

pos Zentropa has much in common with The Third Man, another noir-like film set among the rubble of postwar Europe. Like TTM, there is much inventive camera work. There is an innocent American who gets emotionally involved with a woman he doesn't really understand, and whose naivety is all the more striking in contrast with the natives.<br /><br />But I'd have to say that The Third Man has a more well-crafted storyline. Zentropa is a bit disjointed in this respect. Perhaps this is intentional: it is presented as a dream/nightmare, and making it too coherent would spoil the effect. <br /><br />This movie is unrelentingly grim--"noir" in more than one sense; one never sees the sun shine. Grim, but intriguing, and frightening.


### Tokenizer and Vocab

In [3]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

/Users/mkhokhlush/github/pytorch-ml-experiments/.venv/lib/python3.10/site-packages/torch/utils/data/datapipes/iter/combining.py:180: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


In [4]:
print(tokenizer("here is an example the fdfaf"))
print(vocab(['here', 'is', 'an', 'example', 'the', 'fdfaf']))
vocab.get_itos()[:10]

['here', 'is', 'an', 'example', 'the', 'fdfaf']
[131, 9, 40, 464, 1, 0]


['<unk>', 'the', '.', ',', 'and', 'a', 'of', 'to', "'", 'is']

In [5]:
text_pipeline = lambda line: vocab(tokenizer(line))
label_pipeline = lambda label: 0 if label == "neg" else 1

for label, line in train_iter:
    print(text_pipeline(line))
    print(label_pipeline(label))
    break

[13823, 51, 80, 11, 1124, 19, 1, 851, 134, 3, 163, 47245, 22, 292, 768, 1, 15258, 6, 13682, 2277, 2, 44, 96220, 3, 45, 9, 80, 4384, 390, 169, 2, 45, 9, 40, 1321, 324, 41, 215, 2106, 565, 19, 5, 247, 29, 158, 8, 26, 71, 387, 3, 4, 611, 14584, 9, 36, 1, 58, 3286, 11, 2235, 19, 1, 5844, 2, 21, 12, 8, 244, 32, 7, 140, 14, 1, 851, 134, 51, 5, 58, 10969, 752, 2, 13823, 9, 5, 229, 4154, 11, 13, 1158, 2, 379, 13, 9, 6684, 10, 9, 1316, 17, 5, 65621, 3, 4, 256, 10, 105, 4055, 66, 2316, 1, 937, 2, 13, 20, 9, 24818, 71001, 11, 58, 78, 34, 285, 34, 118, 1056, 1, 2643, 4008, 2, 2582, 3, 21, 1746, 3, 4, 2491, 2]
1


### Dataloader

In [6]:
import torch
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.float).view(-1, 1)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

### Model

In [7]:
from torch import nn

class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, 1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize).to(device)
vocab_size

100683

### Training

In [8]:
import time
import torch.nn.functional as F

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        probability = model(text, offsets)
        loss = criterion(probability, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        # predicted_label = 1 if probability >= 0.5 else 0
        predicted_label = (F.sigmoid(probability) >= 0.5).type(torch.int64)  
        total_acc += (predicted_label == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            probability = model(text, offsets)
            loss = criterion(probability, label)
            predicted_label = (F.sigmoid(probability) >= 0.5).type(torch.int64)  
            total_acc += (predicted_label == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [9]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

train_iter, test_iter = IMDB()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for d in train_dataloader:
    # print(d)
    break

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

/Users/mkhokhlush/github/pytorch-ml-experiments/.venv/lib/python3.10/site-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


-----------------------------------------------------------
| end of epoch   1 | time:  7.55s | valid accuracy    0.707 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   2 | time:  7.68s | valid accuracy    0.819 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   3 | time:  7.47s | valid accuracy    0.778 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   4 | time:  7.38s | valid accuracy    0.842 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   5 | time:  7.25s | valid accuracy    0.838 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   6 | time:  7.32s |

In [10]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.841


In [11]:
ex_text_str = "The movie started cool, but went downhill close to the end."
text = torch.tensor(text_pipeline(ex_text_str))
model = model.to("cpu")
print("Positive probabiluty", model(text, torch.tensor([0]))[0].item())

Positive probabiluty -0.057615041732788086
